In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import models
from tensorflow.keras.layers import Input, Dropout, Dense, Flatten, Input, Layer, concatenate
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import cv2
import gc
import imageio
from os import listdir
from os.path import isfile, join
from tensorflow import keras
import shutil
import os
import PIL.Image
import glob

In [ ]:
# Loading csv files
train_csv = pd.read_csv('../input/petfinder-pawpularity-score/train.csv',index_col=[0])
test_csv = pd.read_csv('../input/petfinder-pawpularity-score/test.csv',index_col=[0])

In [ ]:
# Loading folders containing images 
train_images_folder = "../input/petfinder-pawpularity-score/train/"
test_images_folder = "../input/petfinder-pawpularity-score/test/"

In [ ]:
# Dividing the data
x_train = train_csv.iloc[:,:-1]
y = train_csv.iloc[:,-1]
y = y.sort_index()
x_train = x_train.sort_index()
type(y)

In [ ]:
# Spliting training and validation set from csv file for xgboost model
x_train_1, x_valid, y_train, y_valid = train_test_split(x_train, y, test_size=0.4, random_state=0)

In [ ]:
# Getting a validation array
new_x_valid = np.array(x_valid)

In [ ]:
model_2 = load_model('../input/model-2-1/model_2_1')

In [ ]:
# Compiling the Model_1
model_2.compile(optimizer='adam',loss='mse')

In [ ]:
# Image Data Generator
img_data_gen = ImageDataGenerator(rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.1,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  vertical_flip=True)

In [ ]:
# Defining a function for getting specific images and their outputs
def listing_function(y, x_train_1, images_folder):
    expectations_from_model_1 = []
    imagees = []
    answers = []
    train_images_folder_ready = glob.glob(images_folder + '/*.jpg')
    train_images_folder_ready.sort(reverse=False)

    for imagess in train_images_folder_ready:
            m = (os.path.basename(imagess))
            verify = (m)
            verify = os.path.splitext(verify)
            verify = verify[0]
            for i in x_train_1.index:
                    if verify == i:
                            x_lt = tf.io.read_file(imagess)
                            x_lt = tf.image.decode_jpeg(x_lt, channels=3)
                            x_lt = tf.cast(x_lt, tf.float32) / 255.0
                            x_lt = tf.image.resize(x_lt, (100,100))
                            x_lt = np.array(x_lt)
                            imagees.append(x_lt)
                            
                            x_vals_final = x_train_1.loc[i]
                            x_vals_final = np.array(x_vals_final)
                            expectations_from_model_1.append(x_vals_final)
                            
                            if len(y)>=1:
                                y_final = y[i]
                                y_final = np.array(y_final)
                                answers.append([y_final])
                            else:
                                answers = []

    return expectations_from_model_1, imagees, answers

In [ ]:
# Getting values from the defined function
expectations_from_model_1, imagees, answers = listing_function(y_train, x_train_1, train_images_folder)

In [ ]:
# Converting the values into an array
imagees = np.array(imagees)

In [ ]:
del answers
gc.collect()

In [ ]:
# Training model_1
recall = keras.callbacks.EarlyStopping(monitor='loss',patience=10,restore_best_weights=True)
model_2.fit(img_data_gen.flow(imagees, expectations_from_model_1,
                              batch_size=100),steps_per_epoch=len(x_train_1)//100,
                                                epochs=2, callbacks=[recall])

In [ ]:
del imagees, expectations_from_model_1
gc.collect()

In [ ]:
# Getting test values from the function
req, valid_imgs, valid_answers = listing_function(y_valid, x_valid, train_images_folder)

In [ ]:
# Converting the valid images into an array
array_of_valid_imgs = np.array(valid_imgs)

In [ ]:
# Using the model_1 to predict the features of the pets
predict_test_vals = model_2.predict(array_of_valid_imgs)

In [ ]:
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

In [ ]:
del _,valid_imgs, array_of_valid_imgs
gc.collect()

In [ ]:
def optimizing_for_result(vals):
    object_list = []
    predict_test_vals_1 = vals*10
    for items in predict_test_vals_1:
        element_list = []
        for i in items:
            if i>=3.74:
                i=1
                element_list.append(i)
            else:
                i=0
                element_list.append(i)
        object_list.append(element_list)
    object_list_1 = np.array(object_list)
    return object_list_1

In [ ]:
fitting_res = optimizing_for_result(predict_test_vals)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(req, fitting_res)

In [ ]:
# Building Model_2
xg = XGBRegressor()
xg.fit(req, valid_answers)

In [ ]:
del valid_answers, predict_test_vals#, fitting_res
gc.collect()

# Getting Final results

In [ ]:
_, test_img_final, _ = listing_function([], test_csv, test_images_folder)

In [ ]:
del _
gc.collect()

In [ ]:
array_of_test_img_final = np.array(test_img_final)

In [ ]:
final_features = model_2.predict(array_of_test_img_final)

In [ ]:
final_features_ready = optimizing_for_result(final_features)

In [ ]:
final_features_ready

In [ ]:
test_csv

In [ ]:
result_1 = xg.predict(final_features_ready)

In [ ]:
result_1

In [ ]:
finale = pd.DataFrame()
finale['Id'] = test_csv.index
finale['Pawpularity'] = result_1
finale.to_csv('submission.csv',index=False)